In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("pyspark-sample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d1945472-9484-445b-9c3e-9cd93f365ca8;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.26.30 in central
	found software.amazon.awssdk#aws-xml-protocol;2.26.30 in central
	found software.amazon.awssdk#aws-query-protocol;2.26.30 in central
	found software.amazon.awssdk#protocol-core;2.26.30 in central
	found software.amazon.awssdk#sdk-core;2.26.30 in central
	found software.amazon.awssdk#annotations;2.26.30 in central
	found software.amazon.awssdk#http-client-spi;2.26.30 in central
	found software.amazon.awssdk#utils;2.26.30 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#metric

In [4]:
path = 's3a://raw/datasus/csv/datasus/dbc/*.csv'

In [5]:
df = spark.read.csv(path, header=True, sep=',')

In [6]:
df.count()

240636

In [7]:
df.show(5)

24/08/13 15:18:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------+--------+-----+--------------+-------------+-----+--------+---------+--------+----+----------+----------+----------+----------+---------+----------+----------+----------+----------+---------+----------+----------+----------+--------+-------+--------+------+---------+--------+----------+----------+----------+----------+----------+--------+-------+-------+--------+--------+----------+----------+--------+--------+-------+------+-------+--------+---------+---------+-----+---------+-----+--------+--------+-------+---------+-------+--------+----------+------+---------+----------+----------+---------+------------+--------+------+-----+--------+----------+---------+---------------+---------+-------+--------------+--------+--------+---------+-------+------+-------+-----+--------+-----+---------+--------------------+--------+--------+----------+----------+----------+----------+-------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----

In [8]:
df.columns

['UF_ZI',
 'ANO_CMPT',
 'MES_CMPT',
 'ESPEC',
 'CGC_HOSP',
 'N_AIH',
 'IDENT',
 'CEP',
 'MUNIC_RES',
 'NASC',
 'SEXO',
 'UTI_MES_IN',
 'UTI_MES_AN',
 'UTI_MES_AL',
 'UTI_MES_TO',
 'MARCA_UTI',
 'UTI_INT_IN',
 'UTI_INT_AN',
 'UTI_INT_AL',
 'UTI_INT_TO',
 'DIAR_ACOM',
 'QT_DIARIAS',
 'PROC_SOLIC',
 'PROC_REA',
 'VAL_SH',
 'VAL_SP',
 'VAL_SADT',
 'VAL_RN',
 'VAL_ACOMP',
 'VAL_ORTP',
 'VAL_SANGUE',
 'VAL_SADTSR',
 'VAL_TRANSP',
 'VAL_OBSANG',
 'VAL_PED1AC',
 'VAL_TOT',
 'VAL_UTI',
 'US_TOT',
 'DT_INTER',
 'DT_SAIDA',
 'DIAG_PRINC',
 'DIAG_SECUN',
 'COBRANCA',
 'NATUREZA',
 'NAT_JUR',
 'GESTAO',
 'RUBRICA',
 'IND_VDRL',
 'MUNIC_MOV',
 'COD_IDADE',
 'IDADE',
 'DIAS_PERM',
 'MORTE',
 'NACIONAL',
 'NUM_PROC',
 'CAR_INT',
 'TOT_PT_SP',
 'CPF_AUT',
 'HOMONIMO',
 'NUM_FILHOS',
 'INSTRU',
 'CID_NOTIF',
 'CONTRACEP1',
 'CONTRACEP2',
 'GESTRISCO',
 'INSC_PN',
 'SEQ_AIH5',
 'CBOR',
 'CNAER',
 'VINCPREV',
 'GESTOR_COD',
 'GESTOR_TP',
 'GESTOR_CPF',
 'GESTOR_DT',
 'CNES',
 'CNPJ_MANT',
 'INFEHOSP',
 'C

In [10]:
df.createOrReplaceTempView('tab_sus')

In [17]:
spark.sql("""
    SELECT COUNT(*),
        count(distinct N_AIH)
    FROM tab_sus
""").show()

+--------+---------------------+
|count(1)|count(DISTINCT N_AIH)|
+--------+---------------------+
|  240636|               238970|
+--------+---------------------+



In [24]:
spark.sql("""
    SELECT COUNT(*),N_AIH
    FROM tab_sus
    GROUP BY 2
    ORDER BY 1 DESC
""").show(5)

+--------+-------------+
|count(1)|        N_AIH|
+--------+-------------+
|       5|2324101794883|
|       5|2323106756124|
|       5|2324101826541|
|       5|2323109259383|
|       5|2322107532789|
+--------+-------------+
only showing top 5 rows



In [48]:
spark.sql(f"CREATE DATABASE bronze_datasus LOCATION 's3a://bronze/datasus'")

DataFrame[]

In [49]:
spark.catalog.setCurrentDatabase(f'bronze_datasus')

In [50]:
spark.catalog.listDatabases()

[Database(name='bronze_datasus', catalog='spark_catalog', description='', locationUri='s3a://bronze/datasus'),
 Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/root/.jupyter/workspace/spark-warehouse')]

In [52]:
spark.catalog.currentDatabase()

'bronze_datasus'

In [53]:
df.write.format('delta').mode('overwrite').saveAsTable('bronze_datasus')